In [98]:
import pandas as pd
import numpy as np
from llms import gemini
from llms import chatGPT
import re
import time
x_chat = chatGPT()
x_gemini = gemini()

In [366]:
df = pd.read_csv("../output/data.csv")
df = df[df["num employees"]=="1-10"]
df = df[df["headquarters location"].str.contains("United States")]
df = df[pd.to_datetime(df['founded date'])>'11/30/2022']
sample_df = df.sample(n=10, random_state=1)
sample_df = sample_df.reset_index(drop=True)
sample_df.head()

,Unnamed: 0,organization name,num employees,founded date,description,industries,headquarters location,description.1,cb rank,postal code,...,email,hub tag,phone num,num of sub org,industry groups,total funding amt,company type,estimated rev,description_all,industries_parsed
0,19076,NonprofitsHQ,1-10,2023-02-03,NonprofitsHQ is a powerful suite of AI-powered...,"B2B, SaaS","San Antonio, Texas, United States",B2B SaaS for nonprofits,"476,474",78205,...,hello@nonprofitshq.com,—,—,—,"Other, Software",—,For Profit,—,NonprofitsHQ is a powerful suite of AI-powered...,"['B2B', 'SaaS']"
1,20798,SectorFlow AI,1-10,2023-09-01,—,Artificial Intelligence (AI),"Kings Beach, California, United States",SectorFlow: A cutting-edge platform transformi...,"271,065",—,...,—,—,—,—,"Artificial Intelligence (AI), Data and Analyti...",—,For Profit,—,— SectorFlow: A cutting-edge platform transfor...,[]
2,21223,Abide AI,1-10,2023-10-25,We are trying to simplify challenges across de...,"Artificial Intelligence (AI), Enterprise Softw...","Dover, Delaware, United States",Abide AI is a software development that simpli...,—,19904,...,hello@abideai.com,—,—,—,"Artificial Intelligence (AI), Data and Analyti...",—,For Profit,—,We are trying to simplify challenges across de...,"['Enterprise Software', 'Information Technolog..."
3,21302,inPlace,1-10,2023-11-01,—,"Artificial Intelligence (AI), Furniture, Home ...","New York, New York, United States",inPlace is a furniture rental platform that re...,—,—,...,—,+61 467 666 788,—,—,"Artificial Intelligence (AI), Commerce and Sho...",—,For Profit,$1M to $10M,— inPlace is a furniture rental platform that ...,"['Furniture', 'Home Appliances', 'Rental']"
4,20697,Fownd Inc.,1-10,2023-08-16,FOWND is a company that aims to ease the admin...,"Artificial Intelligence (AI), Health Care, Inf...","Middletown, Delaware, United States",SaaS Platform for Rehabilitation Clinical Docu...,—,19709,...,info@fownd.org,732-983-8218,—,—,"Artificial Intelligence (AI), Data and Analyti...",—,For Profit,—,FOWND is a company that aims to ease the admin...,"['Health Care', 'Information Technology', 'Mac..."


In [ ]:
# df_exp = pd.read_csv('../output/onet/gpt_exposure_embeddings.csv')
# df_exp.head()
# df_exp.description.iloc[0]

In [351]:
generated_description_prompt = """
Your role is to describe $company ($website)'s product. Then, provide a confidence interval on scale on 1-10 on how sure you are about the response.

EXAMPLES BELOW
************************
Company: Genmo
Website: https://www.genmo.ai/
Description: Genmo is a free tool that creates videos and images using artificial intelligence. Genmo to generate creative text formats of text content, like poems, code, scripts, musical pieces, email, letters, etc.

Company: OnePane
Website: https://www.onepane.ai/
Description: Onepane is a company that offers an AI companion for enhanced DevOps & SRE efficiency. Onepane offers a GenAI solution  providing unparalleled unified insights and control over your Cloud resources. Onepane helps with root cause analysis, cloud governance, and optimization strategies.
************************

YOUR TURN:
Company: $company
Wesbite: $website
Current Description: $description
________________________
Description (two sentences):
Confidence Interval:
Reasoning:
________________________

"""



In [352]:
parsed_description_prompt = """
Your role is to describe what jobs/tasks, industries, and customer that $company is targeting. Then, provide a confidence interval (1-10) on scale on 1-10 on how sure you are about the response.

Be specific about people using tool. DO NOT INCLUDE AN EXPLANATION

EXAMPLES BELOW
************************
Company: Petville
Website: petville.co/pricing/biz
Description: Petville Global is a B2B CRM SaaS platform that utilizes advanced technologies like AI/ML and neural net to streamline and expand operations for pet businesses and veterinary clinics both locally and globally. The platform offers deep data analytics and marketing tools, helping businesses save an average of 22% on CRM and vet tech costs.
Tasks/Jobs: Data analysis, Marketing automation, Appointment scheduling, Inventory management
Industry: Customer management
Customers: Vetinarians, Pet Businesses

Company: Thunder
Wesbite: thundercompute.com
Description: Thunder is a decentralized, peer-to-peer cloud computing platform designed to democratize access to GPUs and address the persistent GPU shortage. It provides a solution for individuals and businesses seeking high-performance computing power, enabling them to leverage unused GPU resources from around the world.
Tasks/Jobs: GPU resource allocation, Access to unused GPUs, Distributed computing tasks 
Industry: Cloud Computing
Customers: Developers, GPU Owners

Comapny: NonprofitsHQ
Websit: www.nonprofitshq.com
Description: NonprofitsHQ is a software suite designed for nonprofits that utilizes AI to automate tasks, manage operations, and improve efficiency, ultimately saving organizations time and resources.
Tasks/Jobs: Fundraising management, Donor relationship management, Grant writing, Event planning
Industry: Non-profit management
Customers: Non-profit organizations
************************

YOUR TURN:
________________________
Company: $company
Wesbite: $website
Description: $description
________________________
Tasks/Jobs (comma separated list of 4, short):
Industry (1 item):
Customers (comma separated list): 
________________________
Confidence Interval:
Reasoning:
________________________
"""


In [353]:
examples_prompt = """
Your role is to provide 3 two-sentence examples of how the product from $company might be used. Do not mention the name of the company in the examples, and keep the descriptions broad.

Each example should include:
- A detailed description of the job that the tool automates and who performs that job and when.
- The ONET job being automated
- A confidence interval (1-10) indicating how sure you are about the accuracy of your response.

The goal is to map each example back to ONET jobs. If the job being automated is recognized by ONET, please use the ONET job title. 
If it is not typically found in ONET, use best judgement!

EXAMPLES BELOW: 
************************

Company: HireGuardian
Website: www.hireguardian.com
HireGuardian is a comprehensive workforce management solution that combines employee feedback, background checks, and AI-powered insights to help businesses make informed hiring decisions and build a stronger workforce.  This platform empowers organizations to effectively evaluate potential hires and gain valuable feedback from departing employees, optimizing talent acquisition and retention. 
Tasks/Jobs: Background checks, Employee feedback analysis, Talent acquisition, Workforce management
Industry: Human Resources
People Using Tool: Recruiters, HR professionals 
Example 1: A recruiter uses the platform to automate the process of screening candidates by analyzing their resumes and social media profiles, identifying potential red flags and recommending qualified individuals for further consideration.
ONET JOB automated 1: Human Resource Managers that screen candidates by analyzing their resumes and social media profiles. 
ONET JOB: Human Resource Managers
Example 2: An HR manager uses the platform to conduct exit interviews with departing employees, gathering valuable insights into reasons for leaving and areas for improvement within the company. This feedback helps inform future hiring and retention strategies.
ONET JOB automated 2: Human Resource Managers that conduct manual exit interviews and analyzes responses to identify trends. 
ONET JOB: Human Resource Managers
Example 3: A company utilizes the platform to monitor employee sentiment and identify potential issues before they escalate into larger problems. By analyzing feedback and identifying patterns, they can proactively address concerns and promote a positive work environment.
ONET JOB automated 3: Human Resource Managers that monitor employee sentiment and identifies potential issues manually. 
ONET JOB: Human Resource Managers


Company: Aether
Website: aetherenergie.com/
Aether Energy is an AI-driven platform designed to simplify the process of rooftop solar installation for businesses, providing comprehensive support from project planning and financing to installation and ongoing maintenance. This platform aims to streamline and optimize the entire solar energy journey for installers. 
Example 1: A solar installer uses Aether to quickly create detailed project plans for rooftop solar installations, including system size, panel placement, and wiring diagrams. 
ONET JOB automated 1: Solar Photovoltaic Installers that plan, design, and visualize the installation of a rooftop solar panel.
ONET JOB: Solar Photovoltaic Installers
Example 2: A business owner leverages Aether to secure financing for their rooftop solar project, providing them with customized loan options and streamlined application processes. 
ONET JOB automated 2:  Solar Photovoltaic Installers that research financing options for solar panel installation.
ONET JOB: Solar Photovoltaic Installers
Example 3: A solar installer uses Aether to manage the installation process, tracking materials, scheduling technicians, and coordinating with subcontractors, ensuring smooth project execution. 
ONET JOB automated 3:  A solar installer manages the installation of the solar panels, including tracking materials, scheduling technicians, and coordinating with subcontractors.
ONET JOB: Solar Energy Installation Managers
************************

YOUR TURN:
Company:$company
Website: $website
Current Description: $description
$parsed_description
________________
Example 1: 
ONET JOB automated 1:
ONET JOB 1: 
Confidence Interval 1:
Reasoning 1:

Example 2: 
ONET JOB automated 2:
ONET JOB automated 2: 
Confidence Interval 2:
Reasoning 2:

Example 3:
ONET JOB automated 3:
ONET JOB 3: 
Confidence Interval 3:
Reasoning 3:
________________
"""


In [373]:
class prompting():
    def __init__(self):
        pass
    def iterate(self, df, prompt_template, args, cols, extract_data, i=0):
        results_df = pd.DataFrame(columns=["organization name"]+cols)
        
        for i, row in list(df.iterrows())[i:]:
            prompt = prompt_template
            while True:
                try:
                    name = row['organization name']
                    website = row['website']
                    print(f"Processing {i}: {name}, {website}")
                    if "generated_description" in df.columns:
                        print(row["generated_description"])
                    
                    for arg in args: 
                        prompt = prompt.replace(f"${arg[0]}", row[arg[1]])
                    result = x_gemini.ask(prompt)
                    
                    if result == "N/A": continue #explicit material
                        
                    result_vals = extract_data(result)
                    if result_vals == {}: 
                        print("BAD")
                        print(result)
                        break
                        
                    v = result.replace("\n\n","\n")
                    print(f"Result:\n {v}\n")
                    result_vals["organization name"] = name
                    results_df.loc[i] = dict(result_vals)
                    break 
                
                except Exception as e:
                    print(f"Error processing {index}, {row['organization name']}: {e}")
                    time.sleep(20)  
        for col in cols: 
            if col in df.columns: df = df.drop(columns=cols)
            
        self.results_df = results_df
        df = df.merge(self.results_df, on='organization name', how='left')
        return df

prompting_class = prompting()


In [374]:
def extract_data(text):
    text = text.replace("\n"," ").replace("  "," ").replace("*","").replace(" (two sentences)","").replace("Description:","").replace("/10","").replace("_","").replace("#","")
    pattern = r"^(.*?)\s*Confidence Interval:\s*(\d+)\s*Reasoning:\s*(.*)$"
    match = re.search(pattern, text, re.DOTALL)
    
    if match:
        description = match.group(1).strip()
        confidence_interval = match.group(2).strip()
        reasoning = match.group(3).strip()
        
        result = {
            "generated_description": description,
            "generated_description_conf_interval": confidence_interval,
            "generated_description_conf_interval_reasoning": reasoning
        }
        return result
        
    else:
        return {}
        
args = [["company","organization name"],["website","website"], ["description","description_all"]]
cols = ["generated_description", "generated_description_conf_interval", "generated_description_conf_interval_reasoning"]
sample_df = prompting_class.iterate(sample_df, generated_description_prompt, args, cols, extract_data)


Processing 0: NonprofitsHQ, www.nonprofitshq.com
Result:
 ## NonprofitsHQ Description:
NonprofitsHQ is a comprehensive software platform designed specifically for nonprofits. It offers a suite of AI-powered tools to streamline administrative tasks, manage donor relationships, track fundraising efforts, and improve overall organizational efficiency. The platform aims to empower nonprofits to save time and money, allowing them to focus more on their mission.
## Confidence Interval: 8/10
## Reasoning:
This description is based on an analysis of the NonprofitsHQ website, which highlights the platform's AI capabilities and its focus on simplifying nonprofit operations. While the website doesn't explicitly detail all the specific features, the general focus on streamlining tasks and saving time suggests a broad range of tools for managing various aspects of a nonprofit organization. The "B2B SaaS for nonprofits" designation further reinforces the platform's purpose and target audience. 
Howe

In [322]:
sample_df.head()

,Unnamed: 0,organization name,num employees,founded date,description,industries,headquarters location,description.1,cb rank,postal code,...,num of sub org,industry groups,total funding amt,company type,estimated rev,description_all,industries_parsed,generated_description,generated_description_conf_interval,generated_description_conf_interval_reasoning
0,19076,NonprofitsHQ,1-10,2023-02-03,NonprofitsHQ is a powerful suite of AI-powered...,"B2B, SaaS","San Antonio, Texas, United States",B2B SaaS for nonprofits,"476,474",78205,...,—,"Other, Software",—,For Profit,—,NonprofitsHQ is a powerful suite of AI-powered...,"['B2B', 'SaaS']",NonprofitsHQ is a comprehensive software platf...,8,"The information provided on the website, combi..."


In [378]:
def extract_data(text):
    text = text.replace("\n"," ").replace("  "," ").replace("_","").replace("*","")
    pattern = r".*?Tasks/Jobs:\s*(.*?)\s*Industry:\s*(.*?)\s*Customers:\s*(.*?)\s*Confidence Interval:\s*(\d+)\s*Reasoning:\s*(.*)"
    match = re.search(pattern, text, re.DOTALL)
    
    if match:
        tasks_jobs = match.group(1).strip()
        industry = match.group(2).strip()
        customers = match.group(3).strip()
        confidence_interval = match.group(4).strip()
        reasoning = match.group(5).strip()
        
        result = {
            "parsed_description": tasks_jobs + "\n" + industry + "\n" + customers,
            "Tasks/Jobs": tasks_jobs,
            "Industry": industry,
            "Customers": customers,
            "parsed_description_conf_interval": confidence_interval,
            "parsed_description_conf_interval_reasoning": reasoning
        }
        
        return result
    else:
        print(text)
        return {}
    
args = [["company","organization name"],["website","website"], ["description","generated_description"]]
cols = ["parsed_description", "parsed_description_conf_interval", "parsed_description_conf_interval_reasoning", "Tasks/Jobs","Industry","Customers"]
sample_df = prompting_class.iterate(sample_df, parsed_description_prompt, args, cols, extract_data)



Processing 0: NonprofitsHQ, www.nonprofitshq.com
NonprofitsHQ  NonprofitsHQ is a comprehensive software platform designed specifically for nonprofits. It offers a suite of AI-powered tools to streamline administrative tasks, manage donor relationships, track fundraising efforts, and improve overall organizational efficiency. The platform aims to empower nonprofits to save time and money, allowing them to focus more on their mission.
Result:
 Tasks/Jobs: Fundraising management, Donor relationship management, Grant writing, Event planning
Industry: Non-profit management
Customers: Non-profit organizations, Non-profit leaders, Executive Directors, Board members
Confidence Interval: 9
Reasoning: The website clearly targets non-profit organizations and their leadership, outlining solutions for common tasks like fundraising, donor management, and event planning. 


Processing 1: SectorFlow AI, sectorflow.ai
SectorFlow AI provides an AI-powered platform that analyzes business data, transformi

In [325]:
sample_df.head()

,Unnamed: 0,organization name,num employees,founded date,description,industries,headquarters location,description.1,cb rank,postal code,...,industries_parsed,generated_description,generated_description_conf_interval,generated_description_conf_interval_reasoning,parsed_description,parsed_description_conf_interval,parsed_description_conf_interval_resoning,Tasks/Jobs,Industry,Customers
0,19076,NonprofitsHQ,1-10,2023-02-03,NonprofitsHQ is a powerful suite of AI-powered...,"B2B, SaaS","San Antonio, Texas, United States",B2B SaaS for nonprofits,"476,474",78205,...,"['B2B', 'SaaS']",NonprofitsHQ is a comprehensive software platf...,8,"The information provided on the website, combi...","Fundraising management, Donor relationship man...",9,NaN,"Fundraising management, Donor relationship man...",Non-profit management,"Non-profit organizations, NGOs, Foundations, C..."


In [379]:
def extract_data(text):
    text = text.replace("\n"," ").replace("  "," ").replace("*","").replace("Description:","").replace("/10","").replace("_","").replace("#","")
    pattern = r"Example \d+: (.*?) ONET JOB automated \d+: (.*?) ONET JOB \d+: (.*?) Confidence Interval \d+: (\d+) Reasoning \d+: (.*?)(?= Example \d+:|$)"
    matches = re.findall(pattern, text, re.DOTALL)

    vals = {}
    for i, match in enumerate(matches):
        example_text, onet_job_automated, onet_job, confidence_interval, reasoning = match
        vals[f"Example{i+1}"] = example_text.strip()
        vals[f"Job{i+1}"] = onet_job_automated.strip()
        vals[f"Job{i+1}_title"] = onet_job.strip()
        vals[f"Confidence Interval{i+1}"] = confidence_interval.strip()
        vals[f"Reasoning{i+1}"] = reasoning.strip()
    
    return vals

args = [["company","organization name"],["website","website"], ["description","generated_description"], ["parsed_description","parsed_description"]]
cols = ["situation1", "situation1_conf_interval", "situation1_conf_interval_resoning", "situation2", "situation2_conf_interval", "situation2_conf_interval_resoning","situation3", "situation3_conf_interval", "situation3_conf_interval_resoning",'Example1','Job1','Job1_title','Example2','Job2','Job2_title','Example3','Job3','Job3_title']
sample_df = prompting_class.iterate(sample_df, examples_prompt, args, cols, extract_data)



Processing 0: NonprofitsHQ, www.nonprofitshq.com
NonprofitsHQ  NonprofitsHQ is a comprehensive software platform designed specifically for nonprofits. It offers a suite of AI-powered tools to streamline administrative tasks, manage donor relationships, track fundraising efforts, and improve overall organizational efficiency. The platform aims to empower nonprofits to save time and money, allowing them to focus more on their mission.
Result:
 ## NonprofitsHQ Example Uses
**Example 1:** 
A non-profit development director uses the platform to automate the process of sending personalized thank-you notes to donors after they make a contribution, fostering stronger relationships and encouraging repeat donations. 
**ONET JOB automated 1:**  Fundraising Managers that manually write and send personalized thank-you notes. 
**ONET JOB 1:**  Fundraising Managers 
**Confidence Interval 1:** 9
**Reasoning 1:** This task is a common practice for fundraising managers and aligns with donor relationship

In [ ]:
sample_df.loc[:, ["Job1_ONET", "Job2_ONET", "Job3_ONET"]] = 'N/A'

job_cols = [[f"Job{i}",f"Job{i}_title"] for i in range(1,4)]
embedding_cols = []
for i in range(1,4):  
    embedding_cols += [f"Job{i}_embedding",f"Job{i}_title_embedding"]

results = []
for i, x in sample_df.iterrows():
    name = x["organization name"]
    print(f"Processing {str(i), len(results)}: {name}")
    job_embeddings = [name]
    for job in job_cols: 
        for col in job:
            job_embeddings.append(x_chat.get_embedding(x[col]))
    results.append(job_embeddings)


df = pd.DataFrame(results, columns=['name']+embedding_cols)
sample_df = pd.concat([sample_df, df], axis=1)



Processing ('0', 0): NonprofitsHQ
Processing ('1', 1): SectorFlow AI
Processing ('2', 2): Abide AI
Processing ('3', 3): inPlace
Processing ('4', 4): Fownd Inc.
Processing ('5', 5): DRInclusive


In [348]:
sample_df

,Unnamed: 0,organization name,num employees,founded date,description,industries,headquarters location,description.1,cb rank,postal code,...,Example3,Job3,Job3_title,name,Job1_embedding,Job1_title_embedding,Job2_embedding,Job2_title_embedding,Job3_embedding,Job3_title_embedding
0,19076,NonprofitsHQ,1-10,2023-02-03,NonprofitsHQ is a powerful suite of AI-powered...,"B2B, SaaS","San Antonio, Texas, United States",B2B SaaS for nonprofits,"476,474",78205,...,A nonprofit administrator leverages the platfo...,Nonprofit Administrators who manually compile ...,Non-Profit Administrator,NonprofitsHQ,"[-0.01856156997382641, -0.036197539418935776, ...","[-0.017842872068285942, 0.007264947053045034, ...","[-0.02807483822107315, -0.01150308083742857, -...","[-0.007853823713958263, 0.005379451904445887, ...","[-0.005385094787925482, -0.043300241231918335,...","[-0.03181089833378792, 0.009484394453465939, -..."
